In [ ]:
'''
TrainingBERT on RE:
1) extract all FrontPages from 100 S-3 filings
2) annotate gold examples
3) split 80/20
4) train
PreTraining BERT:
1) collect 10000 filings (take S-3, 8-k with relevant items), strip TitlePage
2) preTrain BERT with Match the Blanks
'''


In [1]:
from pathlib import Path
import logging

root_path = Path(r"C:\Users\Olivi\Desktop\test_set")

In [16]:
# testing annotation process on currently available test set s-3

# parse files with s-3 parser and extract Summary and CoverPage from the the filings
# save to unannoted_s3_cover_page.txt and unannoted_s3_summary.txt
from pathlib import Path
from main.parser.filing_parsers import create_htm_filing
import re
import logging

logger = logging.getLogger()
logger.setLevel(logging.WARNING)
fps = list(Path(r"C:\Users\Olivi\Desktop\test_set\set_s3\filings").rglob("*.htm"))
failed = []
for idx, fp in enumerate(fps[:10]):
    if idx%10 == 0:
        print(f"{idx}|{len(fps)}")
    info = {
        "form_type":"S-3",
        "extension": ".htm",
        "path": fp,
        "filing_date": "-",
        "accession_number": "-",
        "cik": "-",
        "file_number": "-",
    }
    try:
        filings = create_htm_filing(**info)
    except AttributeError as e:
        failed.append((fp, e))
    else:
        if isinstance(filings, list):
            pass
        else:
            filings = [filings]
        for filing in filings:
            for section in filing.sections:
                print(section.title)
                section_text = section.text_only
                if len(section_text) > 100:
                    title = section.title
                    if any([x in title for x in["cover page"]]):
                        title = title[:-1]
                        print(title)
                    # re search in text[:len(title)+1] for a lowercase match and remove by splice if present 
                    
                    print(section_text[:100])
                else:
                    print("-short section")


            # cover_page = filing.get_section(identifier=re.compile("cover"))
            # if cover_page != []:
            #     with open(r"C:\Users\Olivi\Desktop\test_set\unannoted_s3_cover_page.txt", "a", encoding="utf-8") as f:
            #         cover_text = cover_page.text_only
            #         f.write("\n+++::::")
            #         f.write(cover_text)
            # summary = filing.get_section(identifier=re.compile("summary"))
            # if summary != []:
            #     with open(r"C:\Users\Olivi\Desktop\test_set\unannoted_s3_summary.txt", "a", encoding="utf-8") as f:
            #         summary_text = summary.text_only
            #         f.write("\n+++::::")
            #         f.write(summary_text)







0|177
front page
As filed with the Securities and Exchange Commission on December 23, 2020 Registration No. 333- UNIT
cover page 0
cover page 
PROSPECTUS Adams Resources & Energy, Inc. $60,000,000 Common Stock From time to time, we may offer u
toc 0
-short section
about this prospectus
 This prospectus is part of a registration statement that we filed with the Securities and Exchange 
summary
 This summary highlights information contained in other parts of this prospectus or incorporated by 
risk factors
 Investing in our common stock involves a high degree of risk. You should carefully consider the ris
forward-looking statements
 This prospectus and the documents incorporated by reference herein and therein, contain forward-loo
use of proceeds
 We intend to use the net proceeds from the sale of any securities offered under this prospectus for
plan of distribution
 We may sell the securities being offered by us in this prospectus: • directly to purchasers; • thro
description of common 

In [ ]:
# use current SpacyFilingTextSearch to add SECU, SECUQUANTITY, CONTRACT entities aswell as the regular ones: PER, ORG, ect..
# load and split by "+++:::" to get individual pages
with open(root_path / "unannoted_s3_descriptions.txt", "r", encoding="utf-8") as f:
    pages = f.read().split("\n+++::::")
    pages = list(filter(lambda x: len(x)>50, pages))

# load pipeline
from main.nlp.filing_nlp import SpacyFilingTextSearch
import json
logger = logging.getLogger("filing_nlp")
logger.setLevel(logging.INFO)
save_folder = root_path / "s3_doccano_descriptions"
if not save_folder.exists():
    save_folder.mkdir()
search = SpacyFilingTextSearch()
for i, page in enumerate(pages):
    try:
        doc = search.nlp(page)
        # convert to compatible json format for doccano
        # get json first
        j = doc.to_json()
        doccano_dict = {"text": j["text"], "ents": [[i["start"], i["end"], i["label"]] for i in j["ents"]]}
        save_path = save_folder / f"{i}.json"
        with open(save_path, "w", encoding="utf-8") as g:
            json.dump(doccano_dict, g)
    except Exception as e:
        raise e
    


 

In [3]:

from main.nlp.filing_nlp import SpacyFilingTextSearch
import logging
logger = logging.getLogger("filing_nlp")
logger.setLevel(logging.INFO)
search = SpacyFilingTextSearch()

# with open(r"C:\Users\Olivi\Desktop\test_set\unannoted_s3_descriptions.txt", "r", encoding="utf-8") as f:
#     pages = f.read().split("\n+++::::")
#     pages = list(filter(lambda x: len(x)>50, pages))
pages = [
    # "We issued a warrant to purchase up to 6,666,668 shares of common stock in a private placement, dated February 22, 2021.",
    # "The 5000 shares of common stock are comprised of: (1) 2500 shares of common stock issuable upon exercise of Series C Warrants, (2) 2000 shares of common stock issuable upon conversion of Series B Senior Notes and (3) 500 shares of common stock issuable upon conversion of Series K Preferred shares.",
    # "The Series C Preferred Shares are convertible into 500 common shares."
"includes 225,000 shares of our common stock, 168,570 shares of our common stock issuable upon exercise of the Investor Warrant and 155,558 shares of our common stock issuable upon exercise of the Placement Agent Warrant.",
"Placement Agent Warrants for an aggregate amount of 444,444 shares of our common stock were issued to the Placement Agent, Tom Higgins, David Bocchi, Carelo Cataudella, Emanuel Cohen, David Birenbaum, Kevin Oleskewicz, Harry Ioannou, George Anagnostou, Keith Donofrio, Zachart Grodko, and James Tank. The number of shares held by the Placement Agent includes 155,558 shares of our common stock issuable upon exercise of such Placement Agent Warrants.",
"The number of shares consists of 4,444 shares of our common stock issuable upon exercise of the Placement Agent Warrant.",
"The number of shares consists of 65,000 shares of our common stock issuable upon exercise of the Placement Agent Warrant.",
"The number of shares consists of 65,000 shares of our common stock issuable upon exercise of the Placement Agent Warrant.",
"The number of shares consists of 8,000 shares of our common stock issuable upon exercise of the Placement Agent Warrant.",
"The number of shares consists of 3,722 shares of our common stock issuable upon exercise of the Placement Agent Warrant.",
"The number of shares consists of 500 shares of our common stock issuable upon exercise of the Placement Agent Warrant.",
"The number of shares consists of 49,777 shares of our common stock issuable upon exercise of the Placement Agent Warrant.",
"The number of shares consists of 49,777 shares of our common stock issuable upon exercise of the Placement Agent Warrant.",
"The number of shares consists of 21,333 shares of our common stock issuable upon exercise of the Placement Agent Warrant.",
"The number of shares consists of 14,222 shares of our common stock issuable upon exercise of the Placement Agent Warrant.",
"The number of shares consists of 7,111 shares of our common stock issuable upon exercise of the Placement Agent Warrant.",
    ]
docs = []
for idx, page in enumerate(pages):
    if idx%10==0:
        print(f"{idx}/{len(pages)}")
    doc = search.nlp(page)
    docs.append(doc)


    


    # import pickle
    # from spacy.tokens import Span, Token
    # for idx, page in enumerate(pages[:2]):
    #     if idx%10:
    #         print(f"{idx}/{len(pages)}")
    #     doc = search.nlp(page)
#         with open(r"C:\Users\Olivi\Desktop\test_set\desc_test_pickle.txt", "wb") as f:
#             doc._.secu_objects = None
#             doc._.secuquantity_spans = None
#             doc._.secu_objects_map = None
#             doc._.quantity_relation_map = None
#             doc._.secuquantity_secu_map= None
#             doc._.source_quantity_relation_map= None
#             keys_to_remove = []
#             for key in doc.user_data:
#                 if len(key) >= 2:
#                     if key[1] == 'source_span_unmerged':
#                         keys_to_remove.append(key)
#             for key in keys_to_remove:
#                 doc.user_data.pop(key)
#             pickle.dump(doc, f)
#         docs.append(doc)
#     docbin = DocBin(docs=docs, store_user_data=True)
#     docbin.to_disk(r"C:\Users\Olivi\Desktop\test_set\s3_descriptions.spacy")
# else:
#     docbin = DocBin().from_disk(path=r"C:\Users\Olivi\Desktop\test_set\s3_descriptions.spacy")
#     docs = docbin.get_docs(search.nlp.vocab)




# can i hook into msgpack and register my own encoder/decoder for AliasCache and SECUObject ?
# last i could pickle but that would mean way larger files ... 




0/13
includes 225,000 shares of our common stock, 168,570 shares of our common stock issuable upon exercise of the Investor Warrant and 155,558 shares of our common stock issuable upon exercise of the Placement Agent Warrant.
main_secu: common stock
          qrs:[QuantityRelation(                SECUQuantity(225000.0 COUNT -                	 None -                 	 None)                	 common stock)]
None
main_secu: common stock
          qrs:[QuantityRelation(                SECUQuantity(168570.0 COUNT -                	 [issuable] -                 	 None)                	 common stock)]
None
main_secu: Warrant
          qrs:[]
None
main_secu: common stock
          qrs:[QuantityRelation(                SECUQuantity(155558.0 COUNT -                	 [issuable] -                 	 None)                	 common stock)]
None
main_secu: Warrant
          qrs:[]
None
Placement Agent Warrants for an aggregate amount of 444,444 shares of our common stock were issued to the Placement Age

In [3]:

from sdp_utils import *


In [16]:


def is_valid_simple_sdp(sdp_tuple):
    left, right, sdp, ancestor = sdp_tuple
    if len(sdp) < 12:
        if sdp == [*[i for i in left], *[i for i in right]]:
            return False
        sorted_sdp = sorted(sdp, key=lambda x: x.i)
        if left.start == sorted_sdp[0].i and right.start == sorted_sdp[-1].i:
        # if left.start == sorted_sdp[0].i:
            return True
    return False

def secuquant_secu_filter(ents):
    for ent1 in ents:
        for ent2 in ents:
            # if not doesnt_contain_base_alias(ent1, ent2):
            #     continue
            if ent1 != ent2:
                # yield ent1, ent2
                # if ent1.label_ == "SECUQUANTITY" and ent2.label_ == "SECUQUANTITY":
                #     yield ent1, ent2
                # if ent1.label_ == "SECUQUANTITY":
                #     if ent2.label_ == "SECU":
                #         yield ent1, ent2
                # if ent1.label_ == "SECU":
                #     if ent2.label_ == "SECUQUANTITY":
                #         yield ent1, ent2
                if ent1 != ent2:
                    if ent1.label_ == "SECU":
                        if ent2.label_ == "SECU":
                            yield ent1, ent2

# nvu_sdps = get_sdps_noun_verbal(docs, ent_filter=secuquant_secu_filter)
def dont_filter_unwanted_tokens_from_sdp(x):
    return x

sdp_groups = []
for doc in docs:
    sdps, discarded = get_sdps(doc, ent_filter=secuquant_secu_filter, sdp_token_filter=dont_filter_unwanted_tokens_from_sdp, sdp_validation_function=is_valid_simple_sdp)
    for i, x in enumerate(sdps):
        left, right, sdp, ancestor = x
        sorted_sdp = sorted(sdp, key=lambda x: x.i)
        sdp_grouped = group_elements_of_sdp(sorted_sdp)
    # if  all([all([i["name"] in ["noun", "verbal", "unknown"] for i in sdp_grouped]), (len(sdp_grouped) <= 3), ([i["name"] == "verbal" for i in sdp_grouped].count(True) == 1), ([i["name"] == "noun" for i in sdp_grouped].count(True) == 1)]):
        sdp_groups.append((left, right, sdp_grouped, sorted_sdp))
        # print(list(filter(lambda i: [i["name"] == "verbal"], sdp_grouped)))
        # print(left.sent)
        # print(f"{left.text:^20}{right.text:^20} | {sorted_sdp}")
nvu_sdps = sdp_groups


# relations = set()
# verbal_sdps = []
# other_sdps = []
from functools import reduce
def filter_verbal(verbals, sdp_tuple):
    left, right, sdp_grouped, sdp = sdp_tuple
    pos_ = [i.pos_ for i in sdp]
    if "VERB" in pos_ and pos_.count("VERB") == 1:
        verbals.append(sdp_tuple)
    try:
        aux_i = pos_.index("AUX")
        adj_i = pos_.index("ADJ")
    except ValueError:
        pass
    else:
        if (aux_i == adj_i-1) and (sdp[aux_i].i > sdp[adj_i].i - 6):
            verbals.append(sdp_tuple)
    return verbals

def filter_other(other, sdp_tuple):
    left, right, sdp_grouped, sdp = sdp_tuple
    pos_ = [i.pos_ for i in sdp]
    if "VERB" not in pos_ or "AUX" not in pos_:
        print(left, right, pos_, sdp, [p.i for p in sdp], left.sent)
        other.append(sdp_tuple)
    return other


verbal_sdps = reduce(filter_verbal, nvu_sdps, [])
other_sdps = reduce(filter_other, nvu_sdps, [])
print("verbal_sdps:")
for left, right, _, sdp in verbal_sdps:
    print(f"{left.text:20}{right.text:20}|{[sdp]}")
print("other_sdps:")
for left, right, _, sdp in other_sdps:
    print(f"{left.text:20}{right.text:20}|{[sdp]}")
# for left, right, sdp_grouped, sdp in nvu_sdps:
#     if any([token.pos_ == "VERB" for token in sdp]):
#         verbal_sdps.append(sdp)
#     else:
#         other_sdps.append((left, right, sdp_grouped, sdp))
#     info = create_sdp_noun_verbal_dict(left, right, sdp_grouped, sdp)
#     if info["relation"] is not None:
#         relations.add(info["relation"])
    # print(info)
    # print([i["token"] for i in info["sdp"]])
# print(len(verbal_sdps))
# print(len(other_sdps))
    
# for relation in relations:
#     print(relation)
    # print(info["left"], info["right"])
    # print(info["groups"])

# create a file for testing with about 100 sentences with securities and 20 without.
# do i need to account for conj splitting or can i just remove one of the conj since they should refer to the same thing by definition and be resolved by the KB?
# use direct prep_phrases/adverbials for attributes of an entity
# with sdp could create a model to detect wether a valid relation exists (boolean output)
# then build model to classify valid relations with a knowledgebase ?

# first only look at secu and secuquantities and secu secu relations

The number of shares consists of 14,222 shares of our common stock issuable upon exercise of the Placement Agent Warrant.
[14,222, common stock, Warrant]
The number of shares consists of 7,111 shares of our common stock issuable upon exercise of the Placement Agent Warrant.
[7,111, common stock, Warrant]
verbal_sdps:
other_sdps:


In [4]:
def ent_filter(ents):
    for e1 in ents:
        for e2 in ents:
            if e1 != e2:
                yield e1, e2
for doc in docs:
    print("-------")
    print(doc)
    sdps = sdp_between_ents(doc, ent_filter=ent_filter)
    for left, right, sdp, ancestor in sdps:
        print(left, right, sorted(sdp, key=lambda x: x.i), sdp)

# FINAL VERDICT
# DISREGARD SDPS AND JUST MAKE RULES, takes about the same time 
    

-------
includes 225,000 shares of our common stock, 168,570 shares of our common stock issuable upon exercise of the Investor Warrant and 155,558 shares of our common stock issuable upon exercise of the Placement Agent Warrant.
225,000 common stock [225,000, shares, of, common stock] [225,000, shares, of, common stock]
225,000 168,570 [225,000, shares, 168,570, shares] [225,000, shares, shares, 168,570]
225,000 common stock [225,000, shares, shares, of, common stock] [225,000, shares, shares, of, common stock]
225,000 Warrant [225,000, shares, shares, issuable, upon, exercise, of, Warrant] [225,000, shares, shares, issuable, upon, exercise, of, Warrant]
225,000 155,558 [225,000, shares, shares, 155,558, shares] [225,000, shares, shares, shares, 155,558]
225,000 common stock [225,000, shares, shares, shares, of, common stock] [225,000, shares, shares, shares, of, common stock]
225,000 Warrant [225,000, shares, shares, shares, issuable, upon, exercise, of, Warrant] [225,000, shares, sha